In [ ]:
# import mnist library from tensorflow library
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot = True)

# load data 
X_train = mnist.train.images
Y_train = mnist.train.labels

X_test = mnist.test.images
Y_test = mnist.test.labels

batch_X, batch_Y = mnist.train.next_batch(64)

In [ ]:
# simple hello world program
import tensorflow as tf
hello = tf.constant("Hello, World!")

sess = tf.Session()
print(sess.run(hello))

In [ ]:
# basic operations with constants
import tensorflow as tf

a = tf.constant(3)
b = tf.constant(5)

with tf.Session() as sess:
    print "Addition with constants: %i" % sess.run(a+b)
    print "Multiplication with constants: %i " % sess.run(a*b)

In [ ]:
# basic operations with variables
import tensorflow as tf

a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)

add = tf.add(a, b)
mul = tf.multiply(a, b)

with tf.Session() as sess:
    print "Addition with variables %i" % sess.run(add, feed_dict = {a : 3, b : 5})
    print "Multiplication with variables %i" % sess.run(mul, feed_dict = {a : 3, b : 5})

In [ ]:
# basic operations with matrices
import tensorflow as tf

matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[5.], [5.]])

mul = tf.matmul(matrix1, matrix2)

with tf.Session() as sess:
    print "Matrix multiplication %i" % sess.run(mul)

In [ ]:
# nearest neighbors
import numpy as np
import tensorflow as tf

## import mnist data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot = True)

## creating training data
X_tr, Y_tr = mnist.train.next_batch(5000)
## creating test data
X_te, Y_te = mnist.test.next_batch(200)

## creating placeholders
xtr = tf.placeholder("float", [None, 784])
xte = tf.placeholder("float", 784)

## L1 distance
distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices = 1)

pred = tf.arg_min(distance, 0)
accuracy  = 0.

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(len(X_te)):
        nn_index = sess.run(pred, feed_dict = {xtr : X_tr, xte : X_te[i, :]})
        print "Test ", i, "Prediction: ", np.argmax(Y_tr[nn_index]), "True Class: ", np.argmax(Y_te[i])
        if np.argmax(Y_tr[nn_index]) == np.argmax(Y_te[i]):
            accuracy += 1./len(X_te)
    print "Accuracy: ", accuracy

In [ ]:
# linear regression
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt

rng = numpy.random

## parametes
learning_rate = 0.001
training_epochs = 1000
display_step = 50

## training data
train_X = numpy.asarray([3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,7.042,10.791,5.313,7.997,5.654,9.27,3.1])
train_Y = numpy.asarray([1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,2.827,3.465,1.65,2.904,2.42,2.94,1.3])
n_samples = train_X.shape[0]

## placeholders
X = tf.placeholder("float")
Y = tf.placeholder("float")

## coefficients for the linear equation
W = tf.Variable(rng.random(), name = "weight")
b = tf.Variable(rng.random(), name = "bias")

## linear model
pred = tf.add(tf.multiply(X, W), b)
## mean square error
cost = tf.reduce_sum(tf.pow(pred - Y, 2))/(2 * n_samples)

## gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        for (x, y) in zip(train_X, train_Y):
            sess.run(optimizer, feed_dict = {X : x, Y : y})
            
        if (epoch + 1) % display_step == 0:
            c = sess.run(cost, feed_dict = {X : x, Y : y})
            print "Epoch: ", (epoch + 1), "cost: ", c, "weight: ", sess.run(W), "bias: ", sess.run(b)
    plt.plot(train_X, train_Y, 'ro', label='Original data')
    plt.plot(train_X, sess.run(W) * train_X + sess.run(b), label='Fitted line')
    plt.legend()
    plt.show()

In [ ]:
# logistic regression
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

## parameters
learning_rate = 0.001
training_epochs = 25
batch_size = 100
display_step = 1

## training data
X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

## weights and biases
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

## model
pred = tf.nn.softmax(tf.matmul(X, W) + b)
## cost function
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices = 1))
## gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict = {X : batch_xs, y: batch_ys})
            avg_cost += c/batch_size
        print "Epoch: ", epoch, "Cost: ", c
        
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print "Accuracy", accuracy.eval({X : mnist.test.images, y : mnist.test.labels})

In [ ]:
# multilayer perceptron

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

## parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 128

## network parameters
n_hl_1 = 256
n_hl_2 = 256
n_input = 784
n_classes = 10

X = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

## weights 
weights = {
    'h1' : tf.Variable(tf.random_normal([n_input, n_hl_1])),
    'h2' : tf.Variable(tf.random_normal([n_hl_1, n_hl_2])),
    'out' : tf.Variable(tf.random_normal([n_hl_2, n_classes]))
}

## biases
biases = {
    'b1' : tf.Variable(tf.random_normal([n_hl_1])),
    'b2' : tf.Variable(tf.random_normal([n_hl_2])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

## MLP
def multilayer_perceptron(x, weights, biases):
    hl1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    hl1 = tf.nn.relu(hl1)
    
    hl2 = tf.add(tf.matmul(hl1, weights['h2']), biases['b2'])
    hl2 = tf.nn.relu(hl2)
    
    out_layer = tf.matmul(hl2, weights['out']) + biases['out']
    return out_layer

## model
pred = multilayer_perceptron(X, weights, biases)

## cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
## optimizer
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size) 
            _, c = sess.run([optimizer, cost], feed_dict = {X : batch_x, y: batch_y})
            avg_cost += c/total_batch
        print "Epoch: ", epoch, "Cost: ", c
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    print "Accuracy: ", accuracy.eval({X : mnist.test.images, y : mnist.test.labels})

In [11]:
# convolutional neural networks
import tensorflow as tf
from __future__ import print_function
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

## parameters
learning_rate = 0.001
training_epochs = 200000
batch_size = 128

## network parameters
n_input = 784
n_classes = 10
dropout_rate = 0.8

## tf graph input
X = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

## weights
weights = {
    ### convolutional layer 1 - 5*5 convolutional window, 1 input, 32 outputs
    'wc1' : tf.Variable(tf.random_normal([5, 5, 1, 32])),
    ### convolutional layer 2 - 5*5 convolutional window, 32 inputs, 64 outputs
    'wc2' : tf.Variable(tf.random_normal([5, 5, 32, 64])),
    ### fully connected layer - 7*7*64 inputs, 1024 outputs
    'wd1' : tf.Variable(tf.random_normal([7*7*64, 1024])),
    ### output layer - 1024 inputs, 10 output classes
    'out' : tf.Variable(tf.random_normal([1024, 10]))
}

## biases
biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bd1' : tf.Variable(tf.random_normal([1024])),
    'out' : tf.Variable(tf.random_normal([10]))
}

def conv2d(x, w, b, strides = 1):
    x = tf.nn.conv2d(x, w, strides = [1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    x = tf.nn.relu(x)
    return(x)

def maxpool2d(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1, k, k, 1], strides = [1, k, k, 1], padding = 'SAME')

def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape = [-1, 28, 28, 1])

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

pred = conv_net(X, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_epochs:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={X: batch_x, y: batch_y, keep_prob: dropout_rate})
        loss, acc = sess.run([cost, accuracy], feed_dict={X: batch_x, y: batch_y, keep_prob: 1.})
        print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
1
Iter 128, Minibatch Loss= 63693.835938, Training Accuracy= 0.14844
2
Iter 256, Minibatch Loss= 42325.093750, Training Accuracy= 0.14844
3
Iter 384, Minibatch Loss= 34334.757812, Training Accuracy= 0.09375
4
Iter 512, Minibatch Loss= 22935.992188, Training Accuracy= 0.16406
5
Iter 640, Minibatch Loss= 25606.617188, Training Accuracy= 0.09375
6
Iter 768, Minibatch Loss= 28126.500000, Training Accuracy= 0.14844
7
Iter 896, Minibatch Loss= 33322.734375, Training Accuracy= 0.09375
8
Iter 1024, Minibatch Loss= 31564.152344, Training Accuracy= 0.12500
9
Iter 1152, Minibatch Loss= 31607.201172, Training Accuracy= 0.12500
10
Iter 1280, Minibatch Loss= 30712.810547, Training Accuracy= 0.10156
11
Iter 1408, Minibatch Loss= 24786.451172, Training Accuracy= 0.15625
12
Iter 1536, Minibatch Loss= 19328.726562, T